In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

import plotly
plotly.offline.init_notebook_mode(connected=True)

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 10px; background: #DC143C;"><b style="color:white;">The Sharpe Ratio</b></h1></center>

"Sharpe Ratio, developed by Nobel Laureate William F Sharpe (SHA 1966), is a measure of calculating risk adjusted return. It serves to help investors know about the returns on their investments relative to the risks they hold. The Sharpe Ratio is defined as:"

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQf2yxm0EeF3L19YYhSRZIoyfui9DIbzb3CAw&usqp=CAU)temu.land

![](https://i0.wp.com/marketbusinessnews.com/wp-content/uploads/2017/03/Willia-Forsyth-Sharpe-after-whom-Sharpe-ratio-was-named.jpg?resize=931%2C627)marketbusinessnews.com

There should be a balance for image sizes. Not so small but not so huge too like above. 

#Let's take a shot with JPX Tokyo Stock and The Sharpe Ratio

As Katsu1110  San wrote before: This Dataset has so many possibilities to explore. Let's take a shot with JPX Tokyo

In [ ]:
df = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/options.csv", delimiter=',', encoding='ISO-8859-2', parse_dates=['Date'], index_col=['Date'] ).dropna()
pd.set_option('display.max_columns', None)
df.head(2)

#Trading Volume

"Stock price movements aren’t always what they appear. Beneath the surface of a move higher or lower, trends may be forming—or fizzling into a reversal. That’s why traders look for other indications of enthusiasm when they want to confirm or refute a price signal. Trading volume, which measures the number of shares traded during a particular time period, can help."

https://www.schwab.com/resource-center/insights/content/trading-volume-as-market-indicator

#Open interest

"Open interest is the total number of outstanding derivative contracts, such as options or futures that have not been settled for an asset. The total open interest does not count, and total every buy and sell contract. Instead, open interest provides a more accurate picture of the options trading activity, and whether money flows into the futures and options market are increasing or decreasing."

https://www.investopedia.com/terms/o/openinterest.asp

In [ ]:
df1 = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/stock_list.csv", delimiter=',', encoding='ISO-8859-2')
df1.head(2)

#Trading Value

"Trading Value means the average closing sales price, rounded to four decimal points, of the Distributed Securities on the primary securities exchange upon which such Distributed Securities are traded (at the time of the Distribution Demand) for the period of the ten consecutive trading days ending on the second full trading day prior to the Distribution Demand."

https://www.lawinsider.com/dictionary/trading-value#:~:text=Trading%20Value%20means%20the%20average,ending%20on%20the%20second%20full

#Contract month

"The month in which futures contracts may be satisfied by making or accepting a delivery."

https://www.nasdaq.com/glossary/c/contract-month

#Strike price

"A strike price is a set price at which a derivative contract can be bought or sold when it is exercised. For call options, the strike price is where the security can be bought by the option holder; for put options, the strike price is the price at which the security can be sold."

https://www.investopedia.com/terms/s/strikeprice.asp#:~:text=A%20strike%20price%20is%20a,the%20security%20can%20be%20sold.

In [ ]:
df2 = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/trades.csv", delimiter=',', encoding='ISO-8859-2')
pd.set_option('display.max_columns', None)
df2.head(2)

#Session Volume (S_VOL)

"This is the total volume accumulated since the start of the session. For example, in the first 10 minutes of trading the volume is 55 and the second 10 minutes it is 20, the session volume will be 55 for the first bar and 75 for the second bar. This continues to accumulate until the end of the session and then resets to zero at the beginning of the next session. Sessions can be the predefined exchange and primary session or any custom session."

https://www.tradingtechnologies.com/xtrader-help/x-study/technical-indicator-definitions/session-volume-svol/

"What Is a Put-Call Ratio?

"The put-call ratio is a measurement that is widely used by investors to gauge the overall mood of a market."

"A "put" or put option is a right to sell an asset at a predetermined price. A "call" or call option is a right to buy an asset at a predetermined price."

https://www.investopedia.com/ask/answers/06/putcallratio.asp#:~:text=What%20Is%20a%20Put%2DCall,asset%20at%20a%20predetermined%20price.

#We have stock_prices on Train and on Supplemental_files

In [ ]:
#Code by Siddharth Chillaleh https://www.kaggle.com/code/siddalore/sharpe-ratio-study/notebook 

stock_data = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv", delimiter=',', encoding='ISO-8859-2',parse_dates=['Date'], index_col='Date')
stock_data.tail()

#Understanding the Last Trading Day

"The last trading day is the day before a derivative expires. On the expiry date, the derivative is no longer tradable and the settlement process begins. Assume the expiration date on an options contract is Friday, March 22. The last trading is Thursday, March 21."

https://www.investopedia.com/terms/l/lasttradingday.asp#:~:text=The%20last%20trading%20day%20is%20the%20day%20before%20a%20derivative,trading%20is%20Thursday%2C%20March%2021.

In [ ]:
df2 = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv", delimiter=',', parse_dates=['Date'], index_col='Date')
df2.head(2)

#Roll Methodology - Japanese Equity Futures Rolling Strategy Index Methodology 


"Each Japanese Equity Futures Contract expires on a specific quarter end date (one of March, June, 
September or December). At any given time, there is more than one listed Japanese Equity Futures 
Contract on the OSE (for example, with expirations of September 2014, December 2014, March 2015, 
June 2015 and September 2015), and the Index will include the first nearby Japanese Equity Futures 
Contract that has not reached expiration or a roll period (the September 2014 contract in the example)."


"As the first nearby Japanese Equity Futures Contract comes to its expiration, it will be replaced by the 
next Japanese Equity Futures Contract (December 2014 in our example), which is known as the second 
nearby futures contract. Rolling will be carried out during the three exchange business days starting from, 
and including, the third to last exchange business day prior to the special quotation date (as described 
below) of the first nearby Japanese Equity Futures Contract to, and including, the last exchange business 
day prior to the special quotation date of such Japanese Equity Futures Contract. Such period is referred 
to as the “roll period”. 

https://www.goldmansachs.com/what-we-do/global-markets/products-and-business-groups/products/index-methodologies-docs/japanese-index.pdf

In [ ]:
#Code by Clare Tong https://www.kaggle.com/code/claretong/sharpe-ratios-of-tech-stocks/notebook

tech_pivot = df2.pivot_table(index='Date', columns='SupervisionFlag', values='AdjustmentFactor')
tech_final = tech_pivot.dropna(how='any',axis=0)

In [ ]:
#Code by Clare Tong https://www.kaggle.com/code/claretong/sharpe-ratios-of-tech-stocks/notebook

tech_final.info()

In [ ]:
#Code by Clare Tong https://www.kaggle.com/code/claretong/sharpe-ratios-of-tech-stocks/notebook

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

tech_final.plot(title='Historical Stock Prices',linewidth=1);

In [ ]:
#Code by Clare Tong https://www.kaggle.com/code/claretong/sharpe-ratios-of-tech-stocks/notebook

stock_returns = tech_final.pct_change()
stock_returns.plot(subplots=True, linewidth=1);

In [ ]:
#Code by Clare Tong https://www.kaggle.com/code/claretong/sharpe-ratios-of-tech-stocks/notebook

benchmark_returns = tech_final.pct_change()
benchmark_returns.plot(linewidth=1);

#From now, I couldn't get anything else. Sorry for the next lines.

In [ ]:
#Code by Clare Tong https://www.kaggle.com/code/claretong/sharpe-ratios-of-tech-stocks/notebook

excess_returns = stock_returns.sub(benchmark_returns, axis=0)
avg_excess_returns = excess_returns.mean()
plt.subplot(1,2,1) 
avg_excess_returns.plot(kind='bar', title='Mean of the Return Diff');

std_excess_returns = excess_returns.std()
plt.subplot(1,2,2) 
std_excess_returns.plot(kind='bar', title='Std of the Return Diff');
plt.tight_layout()

In [ ]:
#Code by Clare Tong https://www.kaggle.com/code/claretong/sharpe-ratios-of-tech-stocks/notebook

daily_sharpe_ratio = avg_excess_returns.div(std_excess_returns)

annual_factor = np.sqrt(252)
annual_sharpe_ratio = daily_sharpe_ratio.mul(annual_factor)

annual_sharpe_ratio.sort_values().plot(kind='bar', title='Annualized Sharpe Ratio: Tech Stocks vs False',);

In [ ]:
#Code by Clare Tong https://www.kaggle.com/code/claretong/sharpe-ratios-of-tech-stocks/notebook

annual_sharpe_ratio.sort_values()

#Plan B. Spoiler Alert: More snippets failing. Though I'll keep trying Professor Sharpe. 

#Plot & summarize Stock prices 

In [ ]:
#Code by Siddharth Chillaleh https://www.kaggle.com/code/siddalore/sharpe-ratio-study/notebook 

# visualize the stock_data
stock_data.plot(subplots=True, title='Stock Data', figsize=(20, 15));

#The inputs for the Sharpe Ratio

"The Sharpe Ratio uses the difference in returns between the two investment opportunities under consideration."

"This data didn't show the historical value of each investment, not the return. To calculate the return, we need to calculate the percentage change in value from one day to the next. Also take a look at the summary statistics because these will become your inputs as the Sharpe Ratio is calculated."

https://www.kaggle.com/code/siddalore/sharpe-ratio-study/notebook

In [ ]:
#Code by Siddharth Chillaleh https://www.kaggle.com/code/siddalore/sharpe-ratio-study/notebook

#We don't have two different things to compare as Siddharth had. Save this snippet for the Next time

# calculate daily stock_data returns
stock_returns = stock_data.pct_change()

# plot the daily returns
stock_returns.plot(title="stock returns", figsize=(20, 15))

# summarize the daily returns
stock_returns.describe()

In [ ]:
#We don't have two different things to compare as Siddharth had. Save this snippet for the Next time

# calculate the difference in daily returns
excess_returns = stock_returns.sub(sp_returns, axis=0)

# plot the excess_returns
excess_returns.plot( figsize=(20, 15))

# summarize the excess_returns
excess_returns.describe()

"Start computing the Sharpe Ratio. First we need to calculate the average of the excess_returns. This tells us how much more or less the investment yields per day compared to the benchmark."

In [ ]:
#We don't have two different things to compare as Siddharth had. Save this snippet for the Next time

# calculate the mean of excess_returns 
avg_excess_return = excess_returns.mean()

# plot avg_excess_returns
avg_excess_return.plot.bar(title="Mean Of Return difference", figsize=(20, 15))

"Next, calculate the standard deviation of the excess_returns. This shows us the amount of risk an investment in the stocks implies as compared to an investment"

Siddharth Chillaleh https://www.kaggle.com/code/siddalore/sharpe-ratio-study/notebook

In [ ]:
#Code by Siddharth Chillaleh https://www.kaggle.com/code/siddalore/sharpe-ratio-study/notebook

#Save also for next time!!

# calculate the daily sharpe ratio
daily_sharpe_ratio = avg_excess_return.div(sd_excess_return)

# annualize the sharpe ratio
annual_factor = np.sqrt(252)
annual_sharpe_ratio = daily_sharpe_ratio.mul(annual_factor)

# plot the annualized sharpe ratio
annual_sharpe_ratio.plot(figsize=(20, 15));

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSRveH2zLBReSRE6pwhwfbOlVwkWtKEtMmIug&usqp=CAU)twitter.com

#After the snippets above, even for me Professor Sharpe? No need to answer it. It's just a rhetorical question. 